In [189]:
import re
import pandas as pd
import time

dates = ['05-25','05-26','05-27','05-28','05-29','05-30','05-31','06-01','06-02','06-03','06-04','06-05','06-06', '06-07','06-08','06-09','06-10','06-11','06-12','06-13']

url_summary = dict() # format --> 'link': {info}   | {info} format --> {'Datetime', 'Favorites', 'Count'}
total_url_list = [] # ordered from earliest to lastest
double_check_list = []
daily_num_links = [0]

In [190]:
def findUrl(string): 
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"

    url = re.findall(regex,string)       
    return [x[0] for x in url] 

In [191]:
def checkUrl(url_summary, url_list, double_check_list):
    for i in range(len(url_list)-1, -1, -1):
        link = url_list[i]
        if('pic.twitter.com' in link or 'www.instagram.com' in link or 'www.facebook.com' in link or 'SoundCloud.com' in link or 'twitter.com' in link or 'www.amazon.com' in link or 'rddt.co' in link):
            url_list.remove(link)
            url_summary.pop(link)
            # print('removed link: ' + link)

        if('.ly/' in link or '//youtu' in link):
            double_check_list.append(link)
    return url_list

In [192]:
def extractUrl(tweets):
    url_list = []
    for i in range(len(tweets)):
        text = (tweets.loc[i,'Text'])
        urls = findUrl(text)
        if(urls != []):
            for l in urls:
                datetime = tweets.loc[i,'Datetime']
                favs = tweets.loc[i,'Favorites']
                default = {'Datetime': datetime, 'Favs': favs, 'Count': 1}

                if(l in url_summary):
                    url_summary[l]['Favs'] += favs
                    url_summary[l]['Count'] += 1
                else:
                    url_summary[l] = default
                    url_list.append(l)
    return url_list


PROCESS ALL THE TWEETS



In [193]:
for i in range(len(dates)-1):
    tic = time.perf_counter()

    # Preparation
    filename = 'blacklivesmatter_05-25_06-12_topTweets/blacklivesmatter_'+dates[i]+'_toptweets.csv'
    tweets = pd.read_csv(filename)
    r_tweets = tweets[::-1].reset_index() ## reverse the order of the tweets so that it starts with the earliest tweet



    # Processing
    temp_list = extractUrl(r_tweets) # extract all urls without any processing
    temp_list = checkUrl(url_summary, temp_list, double_check_list) # remove irrelevant urls
    total_url_list.extend(temp_list)
    daily_num_links.append(len(temp_list)) # keep track of the number of urls on each day



    toc = time.perf_counter()

    # Brief summary
    daily_total = len(temp_list)
    first_link = temp_list[0]
    last_link = temp_list[-1]
    print('On {}, there are {} links. \n The first link is {} \n the last link is {}'.format(dates[i], daily_total, first_link, last_link))
    print('it took {:0.4f} seconds to complete'.format(toc-tic))
    print('-'*30)

    time.sleep(1)






On 05-25, there are 2 links. 
 The first link is https://vdare.com/posts/this-day-in-vdare-com-history-2015-blacklivesmatter-sometimes-26-shootings-9-killed-in-baltimore-over-memorial-day-weekend 
 the last link is https://www.blackhistory.com/2018/05/black-people-african-americans-invented-memorial-day-history.html
it took 0.0546 seconds to complete
------------------------------
On 05-26, there are 6 links. 
 The first link is https://www.washingtonpost.com/nation/2020/05/26/minneapolis-police-death-custody-fbi/ 
 the last link is https://bit.ly/2X5Hw5O
it took 0.0282 seconds to complete
------------------------------
On 05-27, there are 70 links. 
 The first link is https://tinyurl.com/ycgo2zea 
 the last link is http://chng.it/kqjpwbjj
it took 0.3294 seconds to complete
------------------------------
On 05-28, there are 241 links. 
 The first link is https://www.nbcnews.com/news/us-news/minneapolis-police-officer-center-george-floyd-s-death-had-history-n1215691 
 the last link is h

In [194]:
print(daily_num_links)
print(len(total_url_list))
print(len(url_summary))

for i in total_url_list[-20:]:
    print(i)


[0, 2, 6, 70, 241, 256, 206, 323, 517, 637, 497, 459, 25, 32, 23, 25, 32, 33, 24, 20]
3428
3428
https://scoopusa-pa.newsmemory.com?publink=0cc5f4bbb_13436c5
https://www.albertaparty.ca/racism_in_alberta
https://religionunplugged.com/news/2020/6/11/kenyans-jump-headlong-into-george-floyd-debate-with-mixed-views
https://www.health.com/mind-body/wellness-warriors-black-lives-matter?fbclid=IwAR0TmBGyo7Jm8qRiaNj76q_2KAahuhPME4YZ3JW1gSf8viZSi13gXKM4s_o
https://youtu.be/gXZH-Sz2Ens
https://bit.ly/mteafo
http://ow.ly/SO4G30qOSi8
https://linktr.ee/blacklivesmatter
https://www.phila.gov/2020-06-11-citys-office-of-lgbt-affairs-issues-statement-on-the-murder-of-dominique-remmie-fells/
https://www.london.gov.uk/updates/news-feed
https://bbc.in/3dY3Mon
http://chng.it/BwTnJvwR
http://LovingDay.org
https://www.littlebrown.co.uk/little-brown-news/2020/06/12/read-an-extract-from-tribes-by-david-lammy/
https://crosscut.com/2020/06/black-consultants-accuse-wa-democratic-groups-racism
http://chng.it/qsyRP6

In [195]:
print(len(double_check_list),len(set(double_check_list)))

577 577


In [220]:
import pandas as pd
summary_list = []
for i in total_url_list:
    text = i
    datetime = url_summary[i]['Datetime']
    favs = url_summary[i]['Favs']
    count = url_summary[i]['Count']

    summary_list.append([text, datetime, favs, count])
summary_df = pd.DataFrame(summary_list, columns=['Text', 'Datetime', 'Favs', 'Count'])
summary_df.head()
print(len(summary_df))

3152


In [221]:
summary_df.to_csv('blacklivesmatter_05-25_06-12_topTweets_urlSummary.csv')

In [203]:
len(double_check_list)

577

In [206]:
double_check_df = pd.DataFrame(double_check_list, columns=['Link'])
double_check_df.to_csv('blacklivesmatter_05-25_06-12_topTweets_doubleCheck.csv')

In [209]:
delete_df = pd.read_csv('blacklivesmatter_05-25_06-12_topTweets_deleteLinks.csv')
delete_list = ['blacklivesmatter_05-25_06-12_topTweets_deleteLinks.csv']
for i in range(len(delete_df)):
    delete_list.append(delete_df.loc[i,'Removed Link'])
    

In [210]:
len(delete_list)

277

In [216]:
for i in delete_list:
    if(i in total_url_list and i in url_summary):
        total_url_list.remove(i)
        url_summary.pop(i)

print(len(total_url_list))
print(len(url_summary))

3152
3152
